# TO-DO
1. (DONE) Make config updates recursive. Currently nested dicts are just overwriting. That is, if two config files have a MODEL keyword, the whole model is being overwritten.
2. switch model to `**kwargs`-based setup so we can list all possibly-relevant ones.
3. more-efficient CSV processing

In [3]:
import os
from pathlib import Path
os.chdir(Path(__file__).parent)
    
    
!pip install git+https://github.com/djliden/config-nest.git

    
    
import confignest.confignest
from pathlib import Path
import pandas as pd
import numpy as np
import time
from importlib import import_module

import src.utils.setup
import src.utils.cross_val
import src.utils.eval
import src.utils.metrics
import gc
np.random.seed(623)

NameError: name '__file__' is not defined

In [2]:
model = "fastai_tabular"
configpath = "./src/config/experiments/config_debug.yaml"
default_config = Path("./src/config/default_config.yaml")
cfg = Config(default_config)
model_cfg = Path(f'./src/models/default_configs/{model}.yaml')
model_cls = model.title().replace("_","")
cfg.update_config(model_cfg)
cfg.update_config(configpath)
ct = time.localtime()
current_time = f'{ct[0]}_{ct[1]}_{ct[2]}_{ct[3]}{ct[4]}'
cfg.update_config({'SYSTEM':{'TIME':current_time}})
print(cfg)


CV:
  GAP: 0
  TRAIN_START: 0
  TRAIN_STOP: null
  VAL_END: 210
  VAL_N_ERAS: 3
  VAL_START: 208
DATA:
  REFRESH: false
  SAVE_PROCESSED_TRAIN: true
EVAL:
  CHUNK_SIZE: 1000000
  SAVE_PREDS: false
  SUBMIT_PREDS: false
MODEL:
  BATCHNORM_FINAL: true
  BATCH_SIZE: 2048
  DROPOUT_P: 0
  LAYERS:
  - 400
  - 400
  LEARNING_RATE: 0.001
  LOSS_FUNCTION: MSELossFlat
  METRICS:
  - SpearmanCorrCoef
  NAME: Unnamed
  N_EPOCHS: 1
  TYPE: fastai_tabular
  USE_BATCHNORM: true
  WEIGHT_DECAY: 0
  Y_RANGE:
  - 0
  - 1
SYSTEM:
  DEBUG: false
  TIME: '2021_4_3_823'



In [3]:
credential()
napi = init_numerapi()

Loaded Numerai Public Key into Global Environment!
Loaded Numerai Secret Key into Global Environment!


In [4]:
# 2. Define key paths
round = napi.get_current_round()
#current_file = Path(data_dir/f"numerai_dataset_{round}.zip")
train = Path(f"./input/numerai_dataset_{round}/numerai_training_data.csv")
tourn = Path(f"./input/numerai_dataset_{round}/numerai_tournament_data.csv")
processed = Path('./input/training_processed.csv')
processed_pkl = Path('./input/training_processed.pkl')
output = Path("./output/")

In [5]:
%%time
download_current(napi=napi)
training_data, feature_cols, target_cols = process_current(processed,
                                                           processed_pkl, train, tourn)

The dataset has already been downloaded.
You can re-download it with refresh = True
Loading the pickled training data from file

CPU times: user 127 ms, sys: 1.53 s, total: 1.66 s
Wall time: 3.19 s


time with float16: 1 min 28 s
Time with float32: 1 min 12 s
time with default: 1 min 32 s
time with dask: 56.2 s
time with dask and read train csv with dask: 1.16 s
time deferring compute to after whole concat done: 52.9 s

In [6]:
print(cfg.MODEL)
model_cls

BATCHNORM_FINAL: true
BATCH_SIZE: 2048
DROPOUT_P: 0
LAYERS:
- 400
- 400
LEARNING_RATE: 0.001
LOSS_FUNCTION: MSELossFlat
METRICS:
- SpearmanCorrCoef
NAME: Unnamed
N_EPOCHS: 1
TYPE: fastai_tabular
USE_BATCHNORM: true
WEIGHT_DECAY: 0
Y_RANGE:
- 0
- 1



'FastaiTabular'

In [7]:
# Load Model
modmod = import_module(f'src.models.{model}')
mod = getattr(modmod, model_cls)(**cfg.MODEL.config)

In [8]:
era_split = EraCV(eras = training_data.era)

In [9]:
era_split

EraCV(last era:212)

In [10]:
train_idx, test_idx = era_split.get_splits(valid_start = 209,
                                           valid_n_eras = cfg.CV.VAL_N_ERAS,
                                           train_start = cfg.CV.TRAIN_START,
                                           train_stop = cfg.CV.TRAIN_STOP)

In [11]:
mod.dls = mod.build_data_loaders(df = training_data, cont_names = list(feature_cols),
          train_idx = list(train_idx), val_idx = list(test_idx))
mod.init_learner()

In [12]:
print(mod.learner.lr)

0.001


In [13]:
mod.fit(df = training_data, cont_names = list(feature_cols),
          train_idx = list(train_idx), val_idx = list(test_idx))

training the model

[N | train --------- | valid ----------- | corr ------------- | time]
[0, 0.05665916949510574, 0.054120611399412155, 0.004433362185388489, '00:37']


In [14]:
val_preds = mod.predict(training_data.iloc[test_idx])

prediction = val_preds
target = list(training_data.target.iloc[test_idx])
val_era = list(training_data.era[test_idx])
eval_df = pd.DataFrame({'prediction':prediction,
                        'target':target, 'era':val_era})
eval_df
print(sharpe(eval_df))
print(val_corr(eval_df))

0.4429707313231095
0.004436122191511287


In [15]:
%%script false --no-raise-error
# might be necessary
print("training the model\n")
print(("[N | train --------- | valid ----------- |"
       " corr ------------- | time]"))
start = time.time()
with learn.no_bar():
    learn.fit_one_cycle(n_epoch = cfg.TRAIN.N_EPOCHS,
                        wd = cfg.MODEL.WEIGHT_DECAY)
end = time. time()


In [16]:
save_preds(model=mod, chunksize=5e5,
           pred_path = output/"preds.csv",
           feature_cols = feature_cols, tourn_path=tourn)